In [1]:
# Importo las librerias necesarias

import pandas as pd 
import gzip
import json
from pandas import json_normalize
import ast
import re


In [2]:
# DataFrame N. 1 : steam_games.json.gz
# Inicializacion de Variables. Listas para almacenar los diccionarios y los números de línea con errores
datos = []
lineas_con_errores = []

# Contadores para el total de líneas y registros leídos correctamente
total_lineas = 0
registros_correctos = 0

# Lectura del primer archivo comprimido . Abrir el archivo y leerlo línea por línea. 
with gzip.open('steam_games.json.gz', 'rt', encoding='utf-8') as archivo:

    for num_linea, linea in enumerate(archivo, start=1):  # Bucle for para recorrer cada línea del archivo.
        total_lineas += 1
        try:
            # Intentar cargar la línea como JSON
            data = json.loads(linea)
            datos.append(data)
            registros_correctos += 1
        except json.JSONDecodeError:
            try:
                # Si falla como JSON, intentar cargarlo como diccionario
                data = eval(linea)  # Usa eval para interpretar la línea como un diccionario
                if isinstance(data, dict):
                    datos.append(data)
                    registros_correctos += 1
                else:
                    lineas_con_errores.append(num_linea)
            except Exception as e:
                lineas_con_errores.append(num_linea)

# Crear un DataFrame a partir de la lista de diccionarios
df_steam_games = pd.DataFrame(datos)




In [3]:
# Devuelve el encabezado del dataFrame
df_steam_games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Muestra total de Columnas y filas del DataFrame
df_steam_games.shape

(120445, 13)

In [5]:
# Muestra los valores unicos de la columna 'release_date'
df_steam_games['release_date'].unique()

array([nan, '2018-01-04', '2017-07-24', ..., '2016-11-19', 'January 2018',
       '2018-10-01'], dtype=object)

In [6]:
# Descripción básica del DataFrame:
# Obtener información básica sobre el DataFrame
df_steam_games.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [7]:
# Descripción estadística del DataFrame:
#print(df_steam_games.describe())


In [7]:
# Eliminar las columnas que no voy a utilizar en el calculo de las funciones solicitadas:  El inplace=True Modifica el DataFrame original directamente
# sin necesidad de asignar el resultado a una nueva variable.

columns_to_drop = ['publisher', 'url', 'tags', 'reviews_url', 'specs', 'price', 'early_access', 'developer']

# Eliminar las columnas del DataFrame
df_steam_games.drop(columns=columns_to_drop, inplace=True)


In [8]:
#Verifico si hay datos nulos
df_steam_games.isnull()

,genres,app_name,title,release_date,id
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
...,...,...,...,...,...
120440,False,False,False,False,False
120441,False,False,False,False,False
120442,False,False,False,False,False
120443,False,False,False,False,False


In [9]:
# Elimina las filas que tengan nulos en todas esas columnas simultáneamente.

df_steam_games.dropna(subset=['genres', 'app_name', 'title', 'release_date', 'id'], how='all', inplace=True)


In [10]:
#Eliminar valores nulos en estas columnas de manera simultanea
df_steam_games = df_steam_games.dropna(subset=['genres', 'app_name', 'title', 'release_date', 'id'])


In [11]:
# Se crea una funcion para modificar el tipo de dato fecha de la columna 'release_date', con una expresion regular 
import re
def estandarizar_fecha(fecha):
    if isinstance(fecha, str):
        # Utilizamos una expresión regular para extraer el año de la cadena
        match = re.search(r'\d{4}', fecha)
        if match:
            year = int(match.group())
            # Devolvemos un objeto datetime con el 1 de enero como fecha predeterminada
            return pd.to_datetime(f'{year}-01-01', format='%Y-%m-%d', errors='coerce')
    # Si no se encuentra un año o el valor no es una cadena, devolvemos NaN
    return pd.NaT


In [12]:
# Aplico la funcion para estandarizar el tipo de campo fecha
df_steam_games['release_date'] = df_steam_games['release_date'].apply(estandarizar_fecha)


In [13]:
# Elimina los valores nulos y faltantes
df_steam_games.dropna(subset=['release_date'], inplace=True)


In [14]:
# Muestra las primeras 10 filas del DataFrame con la columna release_date estandarizada
print(df_steam_games[['release_date']].head(30))


      release_date
88310   2018-01-01
88311   2018-01-01
88312   2017-01-01
88313   2017-01-01
88315   2018-01-01
88316   2018-01-01
88317   2018-01-01
88318   2018-01-01
88319   2018-01-01
88322   2018-01-01
88323   2018-01-01
88324   2018-01-01
88325   2018-01-01
88326   2018-01-01
88327   2018-01-01
88328   2018-01-01
88331   2018-01-01
88333   2017-01-01
88334   2017-01-01
88335   2018-01-01
88337   1997-01-01
88338   1998-01-01
88339   2016-01-01
88341   2018-01-01
88342   2018-01-01
88343   2018-01-01
88344   2018-01-01
88345   2017-01-01
88347   2018-01-01
88348   2018-01-01


In [15]:
# Se crea una funcion para asegurarse que no hallan errores con el Tipo Fecha
def encontrar_errores_en_fecha(df, columna):
    errores = []
    
    for indice, valor in df[columna].items():
        if pd.notna(valor):
            try:
                pd.to_datetime(valor, format='%Y-%m-%d')
            except ValueError:
                errores.append((indice, valor))
    
    return errores

In [16]:
# Se llama la funcion para asegurar si existen errores en el campo'release_date'
errores_en_fecha = encontrar_errores_en_fecha(df_steam_games, 'release_date')

if errores_en_fecha:
    print("Se encontraron errores en las siguientes filas:")
    for indice, valor in errores_en_fecha:
        print(f"Fila {indice}: Valor = {valor}")
else:
    print("No se encontraron errores en la columna 'release_date'.")


No se encontraron errores en la columna 'release_date'.


In [17]:
print(df_steam_games['id'].head()) 


88310    761140
88311    643980
88312    670290
88313    767400
88315    772540
Name: id, dtype: object


In [18]:
# Muestra el tipo de dato de un valor específico en la columna 'id' (por ejemplo, el primer valor)
primer_valor_id = df_steam_games['id'].iloc[10]
tipo_del_primer_valor = type(primer_valor_id)
print(f"El tipo de dato del primer valor en la columna 'id' es: {tipo_del_primer_valor}")

# Muestra el tipo de dato de un valor específico en la columna 'title' (por ejemplo, el primer valor)
primer_valor_title = df_steam_games['title'].iloc[10]
tipo_del_primer_valor = type(primer_valor_title)
print(f"El tipo de dato del primer valor en la columna 'title' es: {tipo_del_primer_valor}")


# Muestra el tipo de dato de un valor específico en la columna 'app_name' (por ejemplo, el primer valor)
primer_valor_app_name = df_steam_games['app_name'].iloc[10]
tipo_del_primer_valor = type(primer_valor_app_name)
print(f"El tipo de dato del primer valor en la columna 'app_name' es: {tipo_del_primer_valor}")


# Muestra el tipo de dato de un valor específico en la columna 'app_name' (por ejemplo, el primer valor)
primer_valor_realese_date = df_steam_games['release_date'].iloc[10]
tipo_del_primer_valor = type(primer_valor_realese_date)
print(f"El tipo de dato del primer valor en la columna 'release_date' es: {tipo_del_primer_valor}")

# Muestra el tipo de dato de un valor específico en la columna 'genres' (por ejemplo, el primer valor)
primer_valor_genres = df_steam_games['genres'].iloc[10]
tipo_del_primer_valor = type(primer_valor_genres)
print(f"El tipo de dato del primer valor en la columna 'genres' es: {tipo_del_primer_valor}")

El tipo de dato del primer valor en la columna 'id' es: <class 'str'>
El tipo de dato del primer valor en la columna 'title' es: <class 'str'>
El tipo de dato del primer valor en la columna 'app_name' es: <class 'str'>
El tipo de dato del primer valor en la columna 'release_date' es: <class 'pandas._libs.tslibs.timestamps.Timestamp'>
El tipo de dato del primer valor en la columna 'genres' es: <class 'list'>


In [19]:
# Define una función para verificar si un valor es un número
def es_numero(valor):
    try:
        int(valor)  # Intenta convertir el valor a un entero
        return True
    except ValueError:
        return False




In [20]:
df_steam_games.head(10)

,genres,app_name,title,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-01,761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-01,643980
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-01-01,670290
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-01-01,767400
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-01,772540
88316,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,2018-01-01,774276
88317,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,SNOW - All Access Pro Pass,2018-01-01,774277
88318,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,SNOW - All Access Legend Pass,2018-01-01,774278
88319,"[Casual, Indie, Racing, Simulation]",Race,Race,2018-01-01,768800
88322,"[Action, Adventure, Casual, Indie, RPG]",Army of Tentacles: (Not) A Cthulhu Dating Sim:...,Army of Tentacles: (Not) A Cthulhu Dating Sim:...,2018-01-01,770380


In [21]:
# Encuentra los valores duplicados en la columna 'id'
duplicados = df_steam_games[df_steam_games.duplicated(subset='id', keep=False)]

# Imprime los valores duplicados
print(duplicados)


          genres                          app_name  \
102204  [Action]  Wolfenstein II: The New Colossus   
102883  [Action]  Wolfenstein II: The New Colossus   

                                   title release_date      id  
102204  Wolfenstein II: The New Colossus   2017-01-01  612880  
102883  Wolfenstein II: The New Colossus   2017-01-01  612880  


In [22]:
# Elimina los valores duplicados de la columna 'id'
df_steam_games = df_steam_games.drop_duplicates(subset='id')


In [23]:
# Usar la función isnull() para identificar los valores nulos
nulos = df_steam_games.isnull()

# Sumar los valores nulos en cada columna
nulos_suma = nulos.sum()

# Mostrar el número de valores nulos en cada columna
print(nulos_suma)

genres          0
app_name        0
title           0
release_date    0
id              0
dtype: int64


In [24]:
df_steam_games.head()

,genres,app_name,title,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-01,761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-01,643980
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-01-01,670290
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-01-01,767400
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-01,772540


In [25]:
# DataFrame 2: user_reviews.gz.json
# Creo una funcion para abrir los archivos JSON comprimido
def leer_user_r(archivo):
    with gzip.open(archivo, 'rt', encoding='utf-8') as file:
        return [ast.literal_eval(line.strip()) for line in file]



In [26]:

# Leer el archivo JSON comprimido y convertirlo en un DataFrame
datos2 = leer_user_r('user_reviews.json.gz')
df_user_reviews = pd.DataFrame(datos2)
print(df_user_reviews)

                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
2              evcentric             http://steamcommunity.com/id/evcentric   
3                  doctr                 http://steamcommunity.com/id/doctr   
4              maplemage             http://steamcommunity.com/id/maplemage   
...                  ...                                                ...   
25794  76561198306599751  http://steamcommunity.com/profiles/76561198306...   
25795           Ghoustik              http://steamcommunity.com/id/Ghoustik   
25796  76561198310819422  http://steamcommunity.com/profiles/76561198310...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                                 re

In [27]:
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [28]:
df_user_reviews.shape

(25799, 3)

In [29]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [30]:
# # Descripción estadística del DataFrame:
#print(df_user_reviews.describe())

In [30]:
# Función para extraer un campo específico
def extract_field(reviews_list, columna):
    campo_list = []
    for review in reviews_list:
        campo = review.get(columna, None)
        campo_list.append(campo)
    return campo_list



In [31]:
# Lista de campos que deseas extraer
campos_a_extraer = ['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']
# Aplicar la función a cada campo y crear nuevas columnas en el DataFrame
for campo in campos_a_extraer:
    new_column_name = campo.capitalize()  # Nombre de la nueva columna
    df_user_reviews[new_column_name] = df_user_reviews['reviews'].apply(lambda x: extract_field(x, campo))


In [32]:
df_user_reviews.head(5)

,user_id,user_url,reviews,Funny,Posted,Last_edited,Item_id,Helpful,Recommend,Review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","[, , ]","[Posted November 5, 2011., Posted July 15, 201...","[, , ]","[1250, 22200, 43110]","[No ratings yet, No ratings yet, No ratings yet]","[True, True, True]",[Simple yet with great replayability. In my op...
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","[, , ]","[Posted June 24, 2014., Posted September 8, 20...","[, , ]","[251610, 227300, 239030]",[15 of 20 people (75%) found this review helpf...,"[True, True, True]",[I know what you think when you see this title...
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...","[, , , , , ]","[Posted February 3., Posted December 4, 2015.,...","[, Last edited December 5, 2015., , , , ]","[248820, 370360, 237930, 263360, 107200, 224500]","[No ratings yet, No ratings yet, No ratings ye...","[True, True, True, True, True, True]",[A suitably punishing roguelike platformer. W...
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...","[, , , , , ]","[Posted October 14, 2013., Posted July 28, 201...","[, , , , , ]","[250320, 20920, 204100, 224600, 207610, 108710]",[2 of 2 people (100%) found this review helpfu...,"[True, True, True, True, True, True]",[This game... is so fun. The fight sequences h...
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...","[3 people found this review funny, 1 person fo...","[Posted April 15, 2014., Posted December 23, 2...","[, , , ]","[211420, 211820, 730, 204300]",[35 of 43 people (81%) found this review helpf...,"[True, True, True, True]","[Git gud, It's like Terraria, you play for 9 h..."


In [33]:
# Eliminar las columnas 'user_url', 'funny' y 'helpful'
columns_to_drop = ['user_url', 'reviews', 'Funny', 'Helpful']
df_user_reviews = df_user_reviews.drop(columns=columns_to_drop)

In [34]:
# Reemplaza los valores vacíos con NaN en la columna 'Last_edited'
df_user_reviews['Last_edited'] = df_user_reviews['Last_edited'].apply(lambda x: pd.NA if not x or all(elem == '' for elem in x) else x)

In [35]:
df_user_reviews.dropna(subset=['Last_edited'], inplace=True)

# Restablece el índice si es necesario
df_user_reviews.reset_index(drop=True, inplace=True)

In [36]:
# Función para extraer todos los años de la cadena
def extract_all_years(posted_text):
    # Utiliza una expresión regular para encontrar todos los años en el texto 'Posted'
    import re
    years = re.findall(r'\d{4}', ' '.join(posted_text))
    return [int(year) for year in years]

# Aplicar la función para extraer todos los años y crear la nueva columna 'Year_Posted'
df_user_reviews['Year_Posted'] = df_user_reviews['Posted'].apply(extract_all_years)

In [37]:
df_user_reviews.drop(columns=['Posted', 'Last_edited'], inplace=True)

In [38]:
df_user_reviews.head()

,user_id,Item_id,Recommend,Review,Year_Posted
0,evcentric,"[248820, 370360, 237930, 263360, 107200, 224500]","[True, True, True, True, True, True]",[A suitably punishing roguelike platformer. W...,"[2015, 2014, 2014, 2014, 2014]"
1,Wackky,"[249130, 207610, 550, 65800, 220]","[True, True, True, True, True]","[This game is Marvellous., It reminds me of th...","[2014, 2012, 2012, 2012, 2012]"
2,76561198077246154,"[440, 304930]","[True, True]","[mt bom, É muito bom]",[2014]
3,PPanther,[211820],[True],[I've always liked games like Terraria with th...,[2014]
4,76561198075277292,"[730, 570]","[True, True]",[-Sees player-Tries shoot at the head-Fails-Di...,"[2015, 2015]"


In [39]:
# DataFrame N. 3: user_items.gz.json
# Leer el tercer archivo JSON comprimido y convertirlo en un DataFrame
datos3 = leer_user_r('users_items.json.gz')
df_users_items= pd.DataFrame(datos3)
print(df_users_items)


                       user_id  items_count           steam_id  \
0            76561197970982479          277  76561197970982479   
1                      js41637          888  76561198035864385   
2                    evcentric          137  76561198007712555   
3                   Riot-Punch          328  76561197963445855   
4                        doctr          541  76561198002099482   
...                        ...          ...                ...   
88305        76561198323066619           22  76561198323066619   
88306        76561198326700687          177  76561198326700687   
88307  XxLaughingJackClown77xX            0  76561198328759259   
88308        76561198329548331            7  76561198329548331   
88309        edward_tremethick            0  76561198331598578   

                                                user_url  \
0      http://steamcommunity.com/profiles/76561197970...   
1                   http://steamcommunity.com/id/js41637   
2                 http://st

In [40]:
df_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [41]:
# Campos dentro de items 
items_fields = ['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']

# Crear un diccionario para almacenar los tipos de datos de cada campo
field_data_types = {field: set() for field in items_fields}

# Iterar a través de las filas del DataFrame y analizar los tipos de datos
for index, row in df_users_items.iterrows():
    for items in row['items']:
        for field in items_fields:
            field_value = items.get(field, None)
            if field_value is not None:
                field_data_types[field].add(type(field_value))

# Imprimir los tipos de datos encontrados para cada campo
for field, data_types in field_data_types.items():
    print(f"Campo '{field}': Tipos de datos encontrados {data_types}")

Campo 'item_id': Tipos de datos encontrados {<class 'str'>}
Campo 'item_name': Tipos de datos encontrados {<class 'str'>}
Campo 'playtime_forever': Tipos de datos encontrados {<class 'int'>}
Campo 'playtime_2weeks': Tipos de datos encontrados {<class 'int'>}


In [42]:
# Lista de campos que deseas extraer
campos_a_extraer = ['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']

# Aplicar la función a cada campo y crear nuevas columnas en el DataFrame
for campo in campos_a_extraer:
    new_column_name = campo.capitalize()  # Nombre de la nueva columna
    df_users_items[new_column_name] = df_users_items['items'].apply(lambda x: extract_field(x, campo))

In [43]:
df_users_items.head()

,user_id,items_count,steam_id,user_url,items,Item_id,Item_name,Playtime_forever,Playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [44]:
# Eliminar las columnas 'items_count', 'steam_id', 'user_url', 'items'
columns_to_drop = ['items_count','steam_id','user_url','items','Playtime_2weeks']
df_users_items = df_users_items.drop(columns=columns_to_drop)

In [45]:
df_users_items.head()

,user_id,Item_id,Item_name,Playtime_forever
0,76561197970982479,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."
1,js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507..."
3,Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,..."
4,doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47..."


In [46]:
# Función para convertir minutos a horas
def minutes_to_hours(minutes_list):
    return [round(minutes / 60, 2) for minutes in minutes_list]

# Aplicar la función y crear la nueva columna 'Hours_Played'
df_users_items['Hours_Played'] = df_users_items['Playtime_forever'].apply(lambda x: minutes_to_hours(x))



In [47]:
df_users_items.head()

,user_id,Item_id,Item_name,Playtime_forever,Hours_Played
0,76561197970982479,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ...","[0.1, 0.0, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 78.8..."
1,js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 3.67, 0.0, 0.0, 0.0, 1.03, 0.0..."
2,evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507...","[15.38, 0.0, 0.0, 2.63, 22.05, 0.0, 1.5, 0.0, ..."
3,Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47...","[18.85, 1.48, 2.97, 1.8, 5.22, 1.55, 0.27, 0.0..."


In [48]:
columns_to_drop = ['Playtime_forever']
df_users_items = df_users_items.drop(columns=columns_to_drop)

In [49]:
df_users_items.head()

,user_id,Item_id,Item_name,Hours_Played
0,76561197970982479,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[0.1, 0.0, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 78.8..."
1,js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0.0, 0.0, 0.0, 3.67, 0.0, 0.0, 0.0, 1.03, 0.0..."
2,evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[15.38, 0.0, 0.0, 2.63, 22.05, 0.0, 1.5, 0.0, ..."
3,Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[18.85, 1.48, 2.97, 1.8, 5.22, 1.55, 0.27, 0.0..."


In [50]:

# Extrae el año de 'release_date'
df_steam_games['release_year'] = df_steam_games['release_date'].dt.year

# Expande la columna 'genres'
df_exploded = df_steam_games.explode('genres')

# Selecciona solo las columnas 'genres', 'release_year' e 'id'
df_final_games1 = df_exploded[['genres', 'release_year', 'id']]

# Imprime el resultado
df_final_games1.head()


,genres,release_year,id
88310,Action,2018,761140
88310,Casual,2018,761140
88310,Indie,2018,761140
88310,Simulation,2018,761140
88310,Strategy,2018,761140


In [51]:

# Crea una lista vacía para almacenar los datos
data = []

# Itera sobre las filas del DataFrame original
for idx, row in df_users_items.iterrows():
    for item_id, hours_played in zip(row['Item_id'], row['Hours_Played']):
        # Añade una nueva fila a la lista de datos
        data.append({'Item_id': item_id, 'Hours_Played': hours_played})

# Crea el DataFrame final a partir de la lista de datos
df_final_item1 = pd.DataFrame(data)

# Imprime el resultado
print(df_final_item1)


        Item_id  Hours_Played
0            10          0.10
1            20          0.00
2            30          0.12
3            40          0.00
4            50          0.00
...         ...           ...
5153204  346330          0.00
5153205  373330          0.00
5153206  388490          0.05
5153207  521570          0.07
5153208  519140          0.05

[5153209 rows x 2 columns]


In [52]:
# Une los DataFrames
df_f1 = pd.merge(df_final_games1, df_final_item1, left_on='id', right_on='Item_id')

# Imprime el resultado
print(df_f1)

         genres  release_year      id Item_id  Hours_Played
0        Action          1997  282010  282010          0.08
1        Action          1997  282010  282010          0.00
2        Action          1997  282010  282010          0.00
3        Action          1997  282010  282010          0.00
4        Action          1997  282010  282010          0.22
...         ...           ...     ...     ...           ...
9980157  Action          2004      80      80          0.00
9980158  Action          2004      80      80          0.00
9980159  Action          2004      80      80          0.00
9980160  Action          2004      80      80          0.15
9980161  Action          2004      80      80          1.65

[9980162 rows x 5 columns]


In [53]:
df_f1.shape

(9980162, 5)

In [54]:
# Agrupamos por 'genres' y 'release_year' y sumamos 'Hours_Played'
df_final_f1 = df_f1.groupby(['genres', 'release_year'])['Hours_Played'].sum().reset_index()


In [55]:
df_final_f1.shape

(343, 3)

In [56]:
df_final_f1.to_csv('funcion_1.csv')

In [57]:
# Funcion 2
# Crea una lista vacía para almacenar los datos
data = []

# Itera sobre las filas del DataFrame original
for idx, row in df_users_items.iterrows():
    for item_id, hours_played in zip(row['Item_id'], row['Hours_Played']):
        # Añade una nueva fila a la lista de datos
        data.append({'user_id': row['user_id'], 'Item_id': item_id, 'Hours_Played': hours_played})

# Crea el DataFrame final a partir de la lista de datos
df_final_item2 = pd.DataFrame(data)

# Imprime el resultado
print(df_final_item2)

                   user_id Item_id  Hours_Played
0        76561197970982479      10          0.10
1        76561197970982479      20          0.00
2        76561197970982479      30          0.12
3        76561197970982479      40          0.00
4        76561197970982479      50          0.00
...                    ...     ...           ...
5153204  76561198329548331  346330          0.00
5153205  76561198329548331  373330          0.00
5153206  76561198329548331  388490          0.05
5153207  76561198329548331  521570          0.07
5153208  76561198329548331  519140          0.05

[5153209 rows x 3 columns]


In [58]:
# Une los DataFrames
df_f2 = pd.merge(df_final_games1, df_final_item2, left_on='id', right_on='Item_id')

# Imprime el resultado
df_f2.head()

,genres,release_year,id,user_id,Item_id,Hours_Played
0,Action,1997,282010,UTNerd24,282010,0.08
1,Action,1997,282010,I_DID_911_JUST_SAYING,282010,0.00
2,Action,1997,282010,76561197962104795,282010,0.00
3,Action,1997,282010,r3ap3r78,282010,0.00
4,Action,1997,282010,saint556,282010,0.22


In [59]:
df_f2.shape

(9980162, 6)

In [60]:
# Definimos las horas en un año
horas_en_un_año = 24 * 365

# Agrupamos por 'genres', 'user_id' y 'release_year' y sumamos 'Hours_Played'
df_agrupar_f2 = df_f2.groupby(['genres', 'user_id', 'release_year'])['Hours_Played'].sum().reset_index()

# Eliminamos los registros que superen las horas en un año
df_agrupar_f2 = df_agrupar_f2[df_agrupar_f2['Hours_Played'] <= horas_en_un_año]


In [61]:
df_agrupar_f2.shape

(3436402, 4)

In [62]:


# Asumiendo que df_final_f2 es tu dataframe original
df_grouped = df_agrupar_f2.groupby(['genres', 'release_year']).agg({'Hours_Played': 'idxmax'})

# Esto te dará un dataframe con los índices de las filas del usuario que tiene más horas jugadas por género y año.
# Para obtener el dataframe final con todos los datos de estas filas, puedes hacer:

df_final_f2 = df_agrupar_f2.loc[df_grouped['Hours_Played']]


In [63]:
df_final_f2.tail()

,genres,user_id,release_year,Hours_Played
3433436,Web Publishing,76561198059330972,2013,1090.45
3434218,Web Publishing,Dark_samaCH_PONY,2014,94.13
3435064,Web Publishing,Xyphien,2015,1183.52
3434016,Web Publishing,76561198146189832,2016,2.02
3433232,Web Publishing,76561198045638432,2017,31.28


In [64]:
df_final_f2.to_csv('funcion_2.csv')

In [65]:
df_steam_games.head()


,genres,app_name,title,release_date,id,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-01,761140,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-01,643980,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-01-01,670290,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-01-01,767400,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-01,772540,2018


In [66]:
df_user_reviews.head()

,user_id,Item_id,Recommend,Review,Year_Posted
0,evcentric,"[248820, 370360, 237930, 263360, 107200, 224500]","[True, True, True, True, True, True]",[A suitably punishing roguelike platformer. W...,"[2015, 2014, 2014, 2014, 2014]"
1,Wackky,"[249130, 207610, 550, 65800, 220]","[True, True, True, True, True]","[This game is Marvellous., It reminds me of th...","[2014, 2012, 2012, 2012, 2012]"
2,76561198077246154,"[440, 304930]","[True, True]","[mt bom, É muito bom]",[2014]
3,PPanther,[211820],[True],[I've always liked games like Terraria with th...,[2014]
4,76561198075277292,"[730, 570]","[True, True]",[-Sees player-Tries shoot at the head-Fails-Di...,"[2015, 2015]"


In [67]:
df_final_games3 = df_steam_games[['id', 'title']]


In [68]:
df_final_games3.head()


,id,title
88310,761140,Lost Summoner Kitty
88311,643980,Ironbound
88312,670290,Real Pool 3D - Poolians
88313,767400,弹炸人2222
88315,772540,Battle Royale Trainer


In [69]:


# Crea una lista vacía para almacenar los datos
data = []

# Itera sobre las filas del DataFrame original
for idx, row in df_user_reviews.iterrows():
    for item_id, recommend, year_posted in zip(row['Item_id'], row['Recommend'], row['Year_Posted']):
        # Añade una nueva fila a la lista de datos
        data.append({
            'Item_id': item_id, 
            'Recommend': recommend, 
            'Year_Posted': year_posted
        })

# Crea el DataFrame final a partir de la lista de datos
df_users_reviews3 = pd.DataFrame(data)

# Imprime el resultado
print(df_users_reviews3)



      Item_id  Recommend  Year_Posted
0      248820       True         2015
1      370360       True         2014
2      237930       True         2014
3      263360       True         2014
4      107200       True         2014
...       ...        ...          ...
13997     730       True         2015
13998  407250       True         2015
13999  225840       True         2015
14000     730       True         2015
14001  250340       True         2015

[14002 rows x 3 columns]


In [70]:
df_users_reviews3.head()

,Item_id,Recommend,Year_Posted
0,248820,True,2015
1,370360,True,2014
2,237930,True,2014
3,263360,True,2014
4,107200,True,2014


In [71]:
df_f3 = pd.merge(df_final_games3, df_users_reviews3, left_on='id', right_on='Item_id')



In [72]:
df_f3.head()

,id,title,Item_id,Recommend,Year_Posted
0,282010,Carmageddon Max Pack,282010,True,2015
1,70,Half-Life,70,True,2013
2,70,Half-Life,70,True,2014
3,70,Half-Life,70,True,2013
4,70,Half-Life,70,True,2015


In [74]:
df_f3.to_csv("funcion_3_4.csv")

In [74]:
# Funcion 5.
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analisis_de_sentimiento(review_list):
    sia = SentimentIntensityAnalyzer()
    # Si la lista está vacía, devolver '1' (neutral)
    if not review_list:
        return 1
    else:
        # Concatenar todas las reseñas en una sola cadena de texto
        reviews = ' '.join(review_list)
        polarity_score = sia.polarity_scores(reviews)
        # Devolver '2' (positivo) si la puntuación de polaridad es mayor que 0.05,
        # '0' (negativo) si es menor que -0.05, y '1' (neutral) en caso contrario
        if polarity_score['compound'] > 0.05:
            return 2
        elif polarity_score['compound'] < -0.05:
            return 0
        else:
            return 1

# Aplicar la función a cada lista de reseñas en el dataframe
df_user_reviews['sentiment_analysis'] = df_user_reviews['Review'].apply(analisis_de_sentimiento)

# Reemplazar la columna 'Review' por 'sentiment_analysis'
df_user_reviews.drop('Review', axis=1, inplace=True)


In [76]:
df_user_reviews.head()

,user_id,Item_id,Recommend,Year_Posted,sentiment_analysis
0,evcentric,"[248820, 370360, 237930, 263360, 107200, 224500]","[True, True, True, True, True, True]","[2015, 2014, 2014, 2014, 2014]",2
1,Wackky,"[249130, 207610, 550, 65800, 220]","[True, True, True, True, True]","[2014, 2012, 2012, 2012, 2012]",2
2,76561198077246154,"[440, 304930]","[True, True]",[2014],1
3,PPanther,[211820],[True],[2014],2
4,76561198075277292,"[730, 570]","[True, True]","[2015, 2015]",2


In [77]:
# Crea una lista vacía para almacenar los datos
data = []

# Itera sobre las filas del DataFrame original
for idx, row in df_user_reviews.iterrows():
    for item_id in row['Item_id']:
        # Añade una nueva fila a la lista de datos
        data.append({
            'Item_id': item_id, 
            'sentiment_analysis': row['sentiment_analysis'], 
        })

# Crea el DataFrame final a partir de la lista de datos
df_users_reviews5 = pd.DataFrame(data)

# Imprime el resultado
print(df_users_reviews5)


      Item_id  sentiment_analysis
0      248820                   2
1      370360                   2
2      237930                   2
3      263360                   2
4      107200                   2
...       ...                 ...
17084    4000                   2
17085  304050                   2
17086  220090                   2
17087  262850                   2
17088  265630                   2

[17089 rows x 2 columns]


In [78]:
df_users_reviews5.head()

,Item_id,sentiment_analysis
0,248820,2
1,370360,2
2,237930,2
3,263360,2
4,107200,2


In [79]:

# Extrae el año de 'release_date'
df_steam_games['release_year'] = df_steam_games['release_date'].dt.year

# Selecciona solo las columnas 'genres', 'release_year' e 'id'
df_final_games5 = df_steam_games[['release_year', 'id']]

# Imprime el resultado
df_final_games5.head()

,release_year,id
88310,2018,761140
88311,2018,643980
88312,2017,670290
88313,2017,767400
88315,2018,772540


In [80]:
df_f5 = pd.merge(df_final_games5, df_users_reviews5, left_on='id', right_on='Item_id')

In [81]:
df_f5.head()

,release_year,id,Item_id,sentiment_analysis
0,1997,282010,282010,2
1,1998,70,70,2
2,1998,70,70,0
3,1998,70,70,2
4,1998,70,70,2


In [86]:
df_f5.to_csv('funcion_5.csv')

In [101]:
df_steam_games.head()

,genres,app_name,title,release_date,id,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-01,761140,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-01,643980,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-01-01,670290,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-01-01,767400,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-01,772540,2018


In [102]:
df_user_reviews.head()

,user_id,Item_id,Recommend,Year_Posted,sentiment_analysis
0,evcentric,"[248820, 370360, 237930, 263360, 107200, 224500]","[True, True, True, True, True, True]","[2015, 2014, 2014, 2014, 2014]",2
1,Wackky,"[249130, 207610, 550, 65800, 220]","[True, True, True, True, True]","[2014, 2012, 2012, 2012, 2012]",2
2,76561198077246154,"[440, 304930]","[True, True]",[2014],1
3,PPanther,[211820],[True],[2014],2
4,76561198075277292,"[730, 570]","[True, True]","[2015, 2015]",2


In [103]:
df_users_items.head()

,user_id,Item_id,Item_name,Hours_Played
0,76561197970982479,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[0.1, 0.0, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 78.8..."
1,js41637,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0.0, 0.0, 0.0, 3.67, 0.0, 0.0, 0.0, 1.03, 0.0..."
2,evcentric,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[15.38, 0.0, 0.0, 2.63, 22.05, 0.0, 1.5, 0.0, ..."
3,Riot-Punch,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,doctr,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[18.85, 1.48, 2.97, 1.8, 5.22, 1.55, 0.27, 0.0..."


In [82]:
# Expande la columna 'genres'
df_exploded = df_steam_games.explode('genres')

# Selecciona solo las columnas 'genres', 'release_year' e 'id'
df_final_games_eda = df_exploded[['genres', 'app_name','release_year','id']]

# Imprime el resultado
df_final_games_eda.head()


,genres,app_name,release_year,id
88310,Action,Lost Summoner Kitty,2018,761140
88310,Casual,Lost Summoner Kitty,2018,761140
88310,Indie,Lost Summoner Kitty,2018,761140
88310,Simulation,Lost Summoner Kitty,2018,761140
88310,Strategy,Lost Summoner Kitty,2018,761140


In [83]:
df_final_games_eda.shape

(71208, 4)

In [84]:
# Crea una lista vacía para almacenar los datos
data = []

# Itera sobre las filas del DataFrame original
for idx, row in df_user_reviews.iterrows():
    for  item_id, recommend, year_posted in zip(row['Item_id'], row['Recommend'], row['Year_Posted']):
        # Añade una nueva fila a la lista de datos
        data.append({
            'user_id': row['user_id'],
            'Item_id': item_id, 
            'Recommend': recommend, 
            'Year_Posted': year_posted,
            'sentiment_analysis': row['sentiment_analysis']
            
        })

# Crea el DataFrame final a partir de la lista de datos
df_users_reviews_eda = pd.DataFrame(data)


In [85]:
df_users_reviews_eda.shape

(14002, 5)

In [86]:
# Funcion 2
# Crea una lista vacía para almacenar los datos
data = []

# Itera sobre las filas del DataFrame original
for idx, row in df_users_items.iterrows():
    for item_id,item_name,hours_played in zip(row['Item_id'],row['Item_name'],row['Hours_Played']):
        # Añade una nueva fila a la lista de datos
        data.append({'user_id': row['user_id'], 'Item_id': item_id,'Item_name': item_name, 'Hours_Played': hours_played})

# Crea el DataFrame final a partir de la lista de datos
df_final_item_eda = pd.DataFrame(data)



In [87]:
df_final_games_eda.head()

,genres,app_name,release_year,id
88310,Action,Lost Summoner Kitty,2018,761140
88310,Casual,Lost Summoner Kitty,2018,761140
88310,Indie,Lost Summoner Kitty,2018,761140
88310,Simulation,Lost Summoner Kitty,2018,761140
88310,Strategy,Lost Summoner Kitty,2018,761140


In [88]:
df_users_reviews_eda.head()

,user_id,Item_id,Recommend,Year_Posted,sentiment_analysis
0,evcentric,248820,True,2015,2
1,evcentric,370360,True,2014,2
2,evcentric,237930,True,2014,2
3,evcentric,263360,True,2014,2
4,evcentric,107200,True,2014,2


In [89]:
df_final_item_eda.shape

(5153209, 4)

In [90]:


# Primera fusión
merged_df1_eda = pd.merge(df_final_games_eda, df_users_reviews_eda, left_on='id', right_on='Item_id', how='inner')



In [92]:
merged_df1_eda.shape

(29493, 9)

In [93]:
import random

# Establecemos la semilla para el generador de números aleatorios
random.seed(0)

# Supongamos que 'merged_df1_eda' es tu dataframe
# merged_df1_eda = ...

# Calculamos el 25% del total de registros
n = int(len(df_final_item_eda) * 0.05)

# Seleccionamos una muestra aleatoria del 25% de los registros
merged_df1_eda_muestra_df = df_final_item_eda.sample(n)

# Ahora 'sample_df' es tu nuevo dataframe con el 25% de los registros de 'merged_df1_eda'


In [94]:
merged_df1_eda_muestra_df.shape

(257660, 4)

In [95]:


# Asumiendo que tus dataframes se llaman merged_df1_eda_muestra_df y df_merged_df1_eda
# Unimos los dataframes por el campo 'Item_id'
df_final_eda = pd.merge(merged_df1_eda_muestra_df, merged_df1_eda, on='Item_id', how='inner')




In [97]:

df_final_eda.to_csv('df_final_eda.csv')